←フォルダアイコンをクリック後、PDF ファイルをディレクトリ直下にアップロードしてください

In [ ]:
#@title ライブラリのインストール
!pip install typing-extensions==4.5.0 kaleido==0.2.1 cohere==4.32 langchain==0.0.330 openai==0.28.1 chromadb==0.4.15 pypdf==3.17.0 tiktoken==0.5.1 gradio==3.23.0

In [ ]:
#@title PDFファイルの読み込み・ベクトルDBへの格納・検索方法の設定
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import os
import fnmatch

# OpenAIのAPIキーを設定
APIKey = ""#@param {type:"string"}
os.environ["OPENAI_API_KEY"] = APIKey

# LLMのモデルを設定
LLM=ChatOpenAI(model_name="gpt-3.5-turbo")

# プロンプトテンプレートの設定
prompt_template = """
あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！
以下の文脈を利用して最後の質問に答えてください。答えを知らない場合は、答えを作成しようとせずに「わかりません」と回答してください。

{context}

質問：{question}
回答：
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    #input_variables=["question"],
    input_variables=["context", "question"]
)

for file in os.listdir('./'):
  if fnmatch.fnmatch(file, '*.pdf'):
    print ("PDF : ",file)
    # PDFファイルの読み込み
    loader = PyPDFLoader(file)
    documents = loader.load()
    # 読み込んだ文章の分割
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # ベクトルDBへの格納
    embeddings = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(texts, embeddings)

# 検索方法の設定
qa = RetrievalQA.from_chain_type(
    llm=LLM,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
#@title プロンプト入力用WebUIの起動
import gradio as gr

# メッセージ送信された時の処理
def msg_submit(history, message):
    history = history + [(message, None)]
    return history, ""

# ボットの回答出力処理
def bot(history):
    query = history[-1][0]
    answer = qa.run(query)
    history[-1][1] = answer
    return history

# UI設定
with gr.Blocks() as webui:
    chatbot = gr.Chatbot([], elem_id="chatbot")

    with gr.Row():
        with gr.Column(scale=0.6):
            msg = gr.Textbox(
                show_label=False,
                placeholder="質問内容を入力後、Enterキーで送信",
            )

    msg.submit(msg_submit, [chatbot, msg], [chatbot, msg]).then(
        bot, chatbot, chatbot
    )

#　UI起動
webui.launch(share=True)